In [1]:
%load_ext autoreload
%autoreload 2

In [91]:
import numpy as np
import os

from keras.callbacks import ModelCheckpoint, Callback
from keras.optimizers import Adam
from keras.utils import to_categorical

from networks.unet import Unet

from utils.batch_generator import BatchGenerator

In [83]:
import pandas as pd

In [84]:
BATCH_SIZE = 1
VAL_BATCH = 5
IMG_ROWS, IMG_COLS = 256, 256
INITIAL_EPOCH = 0
NB_EPOCHS = 3
SAVE_MODEL_DIR = './models/'
RESULTS_FILE = 'results.txt'
STEPS_PER_EPOCH = 2

In [30]:
def train(data_dir, val_dir, results_file, save_model_dir, initial_epoch=0):
    batch_gen = BatchGenerator(
        data_dir=data_dir, val_dir=val_dir, batch_size=BATCH_SIZE
    )
    batch_gen.load_data()
    model = DCGAN(IMG_ROWS, IMG_COLS, batch_gen,
                  save_model_dir, results_file, VAL_BATCH)

    if initial_epoch>0:
        model.load_weights(initial_epoch)

    model.train(initial_epoch, NB_EPOCHS, STEPS_PER_EPOCH)

In [23]:
LUNGS_IMGS = './dataset/lungs_segmentation/lungs_resized/'
LUNGS_MASKS = './dataset/lungs_segmentation/masks_dilated_resized/'

In [24]:
files = pd.read_csv('./dataset/lungs_segmentation/files_names.csv')

In [69]:
files['image_path'] = ['{}{}'.format(LUNGS_IMGS, file_name) for file_name in files.file_name.values]
files['mask_path'] = ['{}{}'.format(LUNGS_MASKS, file_name) for file_name in files.file_name.values]

In [70]:
files.head()

,file_name,image_path,mask_path
0,MCUCXR_0023_0.png,./dataset/lungs_segmentation/lungs_resized/MCU...,./dataset/lungs_segmentation/masks_dilated_res...
1,CHNCXR_0286_0.png,./dataset/lungs_segmentation/lungs_resized/CHN...,./dataset/lungs_segmentation/masks_dilated_res...
2,CHNCXR_0022_0.png,./dataset/lungs_segmentation/lungs_resized/CHN...,./dataset/lungs_segmentation/masks_dilated_res...
3,CHNCXR_0067_0.png,./dataset/lungs_segmentation/lungs_resized/CHN...,./dataset/lungs_segmentation/masks_dilated_res...
4,CHNCXR_0572_1.png,./dataset/lungs_segmentation/lungs_resized/CHN...,./dataset/lungs_segmentation/masks_dilated_res...


In [92]:
batch_gen = BatchGenerator(
    data=files, validate=0.2, batch_size=BATCH_SIZE
)

In [93]:
model = Unet(IMG_ROWS, IMG_COLS, batch_gen,
              SAVE_MODEL_DIR, RESULTS_FILE, VAL_BATCH)
model.train(INITIAL_EPOCH, NB_EPOCHS, STEPS_PER_EPOCH)

Epoch 1/3
5/5 [==============================] - 1s 151ms/step
Epoch 2/3
5/5 [==============================] - 1s 167ms/step
Epoch 3/3
5/5 [==============================] - 1s 178ms/step


In [34]:
train(data_dir, val_data_dir, results_file, save_model_dir, initial_epoch)

/usr/local/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
2018-10-21 01:19:00,357 - INFO - One image IOU: 0.523861


5/5 [==============================] - 2s 434ms/step


2018-10-21 01:20:13,890 - INFO - One image IOU: 0.273555


5/5 [==============================] - 2s 425ms/step


In [ ]:
batch_gen = BatchGenerator(
    data_dir=data_dir, val_data_dir=val_data_dir, batch_size=BATCH_SIZE
)
batch_gen.load_data()
model = Unet.model(IMG_ROWS, IMG_COLS)
model.compile(
    optimizer=Adam(lr=1e-4),
    loss=Unet.loss,
    metrics=[Unet.metric]
)

In [ ]:
train_batch, val_batch = batch_gen.generate_test_batch(VAL_BATCH)

In [ ]:
def IOU_loss(y_true, y_false):
    def IOU_calc(y_true, y_false, smooth=1.):
        y_true_f = y_true.flatten()
        y_false_f = y_false.flatten()
        intersection = np.sum(y_true_f * y_false_f)
        return 2 * (intersection + smooth) / (np.sum(y_true_f) + np.sum(y_false_f) + smooth)

    return 1 - IOU_calc(y_true, y_false)